# PHASE 1: Platform Classification using Supervised Learning Methods - Building Training and Test datasets (part 2 b)

### References:
* https://www.quora.com/What-is-the-best-way-to-create-a-training-set-for-machine-learning/answer/Clem-Wang-1

## Main Python libraries

In [1]:
import os, sys, pathlib
from IPython.display import display, Math, Latex #also '%%latex' magic command
import collections, itertools, operator, re, copy, datetime
import urllib, urllib.request, urllib.parse, dns, ipwhois
import pickle, json, csv, zipfile
import math, random, numpy, scipy, pandas
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import bs4
import nltk, sklearn

#actualcwd = os.getcwd()
#os.chdir(actualcwd)
#print(os.getcwd())

In [2]:
#a simple but not adequate hack to solve:
#-- that this notebook will include any passed module that exists on top of its folder
#-- that my installation of the the nltk will find the correct path to the nlkt_data folder
#
#once this line is run, IT SHOULDN'T RUN AGAIN!, otherwise the cwd will change into something different; 
#it can restored though in different ways, one by using `actualcwd` variable or in some cases shuting down this notebook
#
#
os.chdir('../..')
#print(os.getcwd())
sys.path.append(os.getcwd())
import config.config as config

#print(config.anacondadir)

if os.path.exists(config.anacondadir):
    print('ok')
    nltk.data.path.append(config.anacondadir)

ok


In [3]:
STOP HERE!!

SyntaxError: invalid syntax (<ipython-input-3-bfc7f4b2e6e8>, line 1)

In [4]:
#OBSERVATION!: name of file has been changed to reflect last updated file 
if pathlib.Path(os.getcwd()+'/data/annotatedplatformsphase1_a2.csv').is_file():
    pd_annotated = pandas.read_csv(open(os.getcwd()+'/data/annotatedplatformsphase1_a2.csv', 'r'), sep=';', quotechar="'")
else:
    print('Not path found')

In [5]:
pd_annotated.head(5)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1,alltext,category,category_regex,description,htext,keywords,params,platform,title,wiki
0,0,0,0,components clearfix world carousel buttons h...,PACKAGE,api|package|framework|librar|stack|licens|addo...,"the most popular html, css, and js framework i...","Bootstrap is the most popular HTML, CSS, and J...",,",/components,/components/breadcrumb/,/migratio...",v4-alpha.getbootstrap.com,"\n \n bootstrap · the most popular html, c...",NaN
1,1,3,3,plugin loaded github new com iscroll cubiq e...,PACKAGE,api|package|framework|librar|stack|licens|addo...,,jScroll is a jQuery plugin for infinite scroll...,,"/,https://github.com/cubiq/iscroll",jscroll.com,jscroll - a jquery plugin for infinite scrolli...,NaN
2,2,10,10,validation mongoose models node queries api ...,PACKAGE,api|package|framework|librar|stack|licens|addo...,,Elegant MongoDB object modeling for Node.js Mo...,,"/docs/promises.html,/docs/populate.html,/docs/...",mongoosejs.com,mongoose odm v4.10.5,NaN
3,3,34,34,charts samples jsonp navigator view highchar...,PACKAGE,api|package|framework|librar|stack|licens|addo...,Highcharts - Interactive JavaScript charts for...,View demo Get a license,"highcharts, charts, javascript charts, ajax ch...","/samples/data/jsonp.php,/blog/192-use-highchar...",www.highcharts.com,Interactive JavaScript charts for your webpage...,NaN
4,4,41,41,crossorigin org wikipedia api w free https e...,PACKAGE,api|package|framework|librar|stack|licens|addo...,,"\n Welcome to crossorigin.me,...",,"/https://en.wikipedia.org/w/api.php,/,/https:/...",www.crossorigin.me,crossorigin.me,NaN


In [6]:
#OBSERVATION!: name of file has been changed to reflect last updated file 
if pathlib.Path(os.getcwd()+'/data/notatedplatformsphase1_a2.csv').is_file():
    pd_notated = pandas.read_csv(open(os.getcwd()+'/data/notatedplatformsphase1_a2.csv', 'r'), sep=';', quotechar="'")
else:
    print('Not path found')

In [7]:
pd_notated.head(5)

,Unnamed: 0,platform,title,description,keywords,htext,params,category,wiki
0,0,svenv.nl,\n\n \n Home\n ...,NaN,,"Nowadays, many webpages are being styled using...",/programming/whyihatebootstrap,NaN,NaN
1,1,envato.com,Envato - Top digital assets and services,Join millions & bring your ideas and projects ...,,,/blog/css3-flexbox/,NaN,Envato (formerly Eden) operates a group of dig...
2,2,naimhamadi.wordpress.com,Naim Hammadi's Blog,,,Naim Hammadi's Blog,/2014/05/13/mastering-css-selectors/,NaN,WordPress.com is a blog web hosting service pr...
3,3,www.fusioncharts.com,,,,,/blog/2013/12/jsdoc-vs-yuidoc-vs-doxx-vs-docco...,NaN,"FusionCharts, part of InfoSoft Global (P) Ltd,..."
4,4,lh3.googleusercontent.com,Error 404 (Not Found)!!1,,,404. That’s an error.\n,/MewY8qm8NVCQ3YcSuy92f8sUP18qUfKIQkUJhLq46otOj...,NaN,NaN


## DATA UPDATING - Important...

The code commented below was used to:
* **Add wikipedia info to the created tables**: for this I used the `scr.wikipedia_extract` module made for this purpose
* **Revise some column naming and use the names for categories** to be used from now on: until now (29-July-17), I was using as categories a regex form I used for the original classification rules; from now I will be using the defined categories found in the `docs` folder of this work

The changes below were done by creating updated files, re-opening that file and update it until the right file was created. Once the information was updated with some information, the respective code was commented because it was not of usage any more.

The code below is kept commented but is shown so other users of this file can have a reference. Also the different intermediate files are kept in the `data` folder but only the required files are called for further analysis. In a more advanced phase a simpler file will be kept.

In [ ]:
#import scr.wikipedia_extract

In [ ]:
#import imp

In [ ]:
#imp.reload(scr.wikipedia_extract)

In [ ]:
#pd_annotated['wiki'] = ''
#pd_notated['wiki'] = ''
#scr.wikipedia_extract.getting_wikipedia(pd_annotated)
#scr.wikipedia_extract.getting_wikipedia(pd_notated)

In [ ]:
#pd_errortemp = pd_notated.loc[pd_notated['platform'] == 'view-source:http:',]
#scr.wikipedia_extract.getting_wikipedia(pd_errortemp)
#print(pd_errortemp)
#print(pd_notated.iloc[110,])
#url = 'https://en.wikipedia.org/w/api.php?action=query&list=search&format=json&srsearch=JavaScript Objects in Detail'
#req = urllib.request.Request(url)
#try:
#    resp = urllib.request.urlopen(req)
#except urllib.error.HTTPError as e:
#    print(e)

In [ ]:
#pd_notated.head(100)

In [ ]:
#pd_annotated = pd_annotated.rename(columns={'category':'category_regex'})

In [ ]:
#pd_annotated.columns

In [ ]:
#categories = pandas.read_csv(open(os.getcwd()+'/docs/category_operationalization.csv', 'r'), sep=':')

In [ ]:
#categories

In [ ]:
#categories = categories[['category','category_regex']]

In [ ]:
#pd_annotated = pandas.merge(pd_annotated, categories, on=['category_regex'])

In [ ]:
#pd_annotated.head(5)

In [ ]:
#pd_annotated.to_csv(os.getcwd()+'/data/annotatedplatformsphase1_a1.csv', sep=';', quotechar="'")
#pd_notated.to_csv(os.getcwd()+'/data/notatedplatformsphase1_a2.csv', sep=';', quotechar="'")

In [ ]:
STOP HERE!

## DATA PREPARATION

In [8]:
import scr.datapreparation_ML

ok


In [9]:
#imp.reload(scr.datapreparation_ML)

In [10]:
for col in pd_annotated.columns:
    print(col,' has the following null values: ', pd_annotated[[col]].isnull().sum())

Unnamed: 0  has the following null values:  Unnamed: 0    0
dtype: int64
Unnamed: 0.1  has the following null values:  Unnamed: 0.1    0
Unnamed: 0.1    0
dtype: int64
Unnamed: 0.1  has the following null values:  Unnamed: 0.1    0
Unnamed: 0.1    0
dtype: int64
alltext  has the following null values:  alltext    0
dtype: int64
category  has the following null values:  category    0
dtype: int64
category_regex  has the following null values:  category_regex    323
dtype: int64
description  has the following null values:  description    20
dtype: int64
htext  has the following null values:  htext    0
dtype: int64
keywords  has the following null values:  keywords    5
dtype: int64
params  has the following null values:  params    19
dtype: int64
platform  has the following null values:  platform    0
dtype: int64
title  has the following null values:  title    3
dtype: int64
wiki  has the following null values:  wiki    643
dtype: int64


In [11]:
pd_annotated.shape

(773, 13)

In [12]:
for col in pd_notated.columns:
    print(col,' has the following null values: ', pd_notated[[col]].isnull().sum())

Unnamed: 0  has the following null values:  Unnamed: 0    0
dtype: int64
platform  has the following null values:  platform    0
dtype: int64
title  has the following null values:  title    0
dtype: int64
description  has the following null values:  description    3
dtype: int64
keywords  has the following null values:  keywords    2
dtype: int64
htext  has the following null values:  htext    0
dtype: int64
params  has the following null values:  params    7
dtype: int64
category  has the following null values:  category    258
dtype: int64
wiki  has the following null values:  wiki    210
dtype: int64


In [13]:
pd_notated.shape

(258, 9)

In [14]:
#pd_annotated['alltext'] = ''
#scr.datapreparation_ML.datapreparation(pd_annotated)

In [15]:
pd_notated['alltext'] = ''
scr.datapreparation_ML.datapreparation(pd_notated)

In [16]:
def jscheck(x):
    plat = x
    if len(plat.split('.')) > 1:
        if len(plat.split('.')[-2]) > 5:
            if plat.split('.')[-2][-2:] == 'js':
                return 1
        elif plat.split('.')[0] == 'api':
            return 1
    
    return 0
            

In [17]:
def docscheck(x):
    plat = x
    if len(plat.split('.')) > 1:
        if len(plat.split('.')[0]) > 2:
            if plat.split('.')[0] in ['docs', 'dev', 'developers', 'developer']:
                return 1
    return 0

In [18]:
def blogscheck(x):
    plat = x
    if len(plat.split('.')) > 1:
        if len(plat.split('.')[0]) > 2:
            if plat.split('.')[0] in ['blog', 'blogs', 'articule', 'articules']:
                return 1
    return 0

In [19]:
def commcheck(x):
    plat = x
    if len(plat.split('.')) > 1:
        if len(plat.split('.')[0]) > 2:
            if plat.split('.')[0] in ['forum', 'forums', 'community', 'channel']:
                return 1
    return 0

In [20]:
pd_annotated['jscheck'] = None
pd_annotated['docscheck'] = None
pd_annotated['blogscheck'] = None
pd_annotated['commcheck'] = None

In [21]:
pd_annotated['jscheck'] = pd_annotated['platform'].apply(jscheck)
pd_annotated['docscheck'] = pd_annotated['platform'].apply(docscheck)
pd_annotated['blogscheck'] = pd_annotated['platform'].apply(blogscheck)
pd_annotated['commcheck'] = pd_annotated['platform'].apply(commcheck)

In [22]:
pd_annotated.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1,alltext,category,category_regex,description,htext,keywords,params,platform,title,wiki,jscheck,docscheck,blogscheck,commcheck
0,0,0,0,components clearfix world carousel buttons h...,PACKAGE,api|package|framework|librar|stack|licens|addo...,"the most popular html, css, and js framework i...","Bootstrap is the most popular HTML, CSS, and J...",,",/components,/components/breadcrumb/,/migratio...",v4-alpha.getbootstrap.com,"\n \n bootstrap · the most popular html, c...",NaN,0,0,0,0
1,1,3,3,plugin loaded github new com iscroll cubiq e...,PACKAGE,api|package|framework|librar|stack|licens|addo...,,jScroll is a jQuery plugin for infinite scroll...,,"/,https://github.com/cubiq/iscroll",jscroll.com,jscroll - a jquery plugin for infinite scrolli...,NaN,0,0,0,0
2,2,10,10,validation mongoose models node queries api ...,PACKAGE,api|package|framework|librar|stack|licens|addo...,,Elegant MongoDB object modeling for Node.js Mo...,,"/docs/promises.html,/docs/populate.html,/docs/...",mongoosejs.com,mongoose odm v4.10.5,NaN,1,0,0,0
3,3,34,34,charts samples jsonp navigator view highchar...,PACKAGE,api|package|framework|librar|stack|licens|addo...,Highcharts - Interactive JavaScript charts for...,View demo Get a license,"highcharts, charts, javascript charts, ajax ch...","/samples/data/jsonp.php,/blog/192-use-highchar...",www.highcharts.com,Interactive JavaScript charts for your webpage...,NaN,0,0,0,0
4,4,41,41,crossorigin org wikipedia api w free https e...,PACKAGE,api|package|framework|librar|stack|licens|addo...,,"\n Welcome to crossorigin.me,...",,"/https://en.wikipedia.org/w/api.php,/,/https:/...",www.crossorigin.me,crossorigin.me,NaN,0,0,0,0


In [23]:
pd_notated['jscheck'] = None
pd_notated['docscheck'] = None
pd_notated['blogscheck'] = None
pd_notated['commcheck'] = None

In [24]:
pd_notated['jscheck'] = pd_notated['platform'].apply(jscheck)
pd_notated['docscheck'] = pd_notated['platform'].apply(docscheck)
pd_notated['blogscheck'] = pd_notated['platform'].apply(blogscheck)
pd_notated['commcheck'] = pd_notated['platform'].apply(commcheck)

In [25]:
pd_notated.head()

,Unnamed: 0,platform,title,description,keywords,htext,params,category,wiki,alltext,jscheck,docscheck,blogscheck,commcheck
0,0,svenv.nl,\n\n \n Home\n ...,NaN,,"Nowadays, many webpages are being styled using...",/programming/whyihatebootstrap,NaN,NaN,also nl framework think even us less result ...,0,0,0,0
1,1,envato.com,Envato - Top digital assets and services,Join millions & bring your ideas and projects ...,,,/blog/css3-flexbox/,NaN,Envato (formerly Eden) operates a group of dig...,envato creative flexbox ideas top world serv...,0,0,0,0
2,2,naimhamadi.wordpress.com,Naim Hammadi's Blog,,,Naim Hammadi's Blog,/2014/05/13/mastering-css-selectors/,NaN,WordPress.com is a blog web hosting service pr...,naim hammadi selectors blog css mastering,0,0,0,0
3,3,www.fusioncharts.com,,,,,/blog/2013/12/jsdoc-vs-yuidoc-vs-doxx-vs-docco...,NaN,"FusionCharts, part of InfoSoft Global (P) Ltd,...",doxx generator jsdoc vs blog yuidoc choosing...,0,0,0,0
4,4,lh3.googleusercontent.com,Error 404 (Not Found)!!1,,,404. That’s an error.\n,/MewY8qm8NVCQ3YcSuy92f8sUP18qUfKIQkUJhLq46otOj...,NaN,NaN,rw error h1080 found mewy8qm8nvcq3ycsuy92f8s...,0,0,0,0


In [26]:
STOP HERE!

SyntaxError: invalid syntax (<ipython-input-26-d5af98942240>, line 1)

# ANALYSIS 3

## UTILITIES

In [144]:
from sklearn.feature_extraction.text import CountVectorizer
def vectorizer(documents):
    countvect = CountVectorizer(ngram_range=(1,2))
    X_counts = countvect.fit_transform(documents)
    return X_counts, countvect

from scipy.sparse import hstack
import copy
def hstacker(X, hackedcolumns):
    X_countsh = hstack((copy.copy(X), hackedcolumns))
    return X_countsh


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
def modelling(Classifier, X, y, params):
    #clf_dtm = DecisionTreeClassifier(max_depth=1, min_samples_leaf=1, random_state=0)
    clf_dtm = Classifier().set_params(**params)
    return clf_dtm, clf_dtm.fit(X, y)

def compareyvsypredict(target, y, y_predict):
    print('{0:35} => {1:15} => {2:15}'.format('Target','Class','PredClass'))
    for plat, category in zip(zip(target, y), y_predict):
        print('{0:35} => {1:15} => {2:15}'.format(plat[0], plat[1], category))
    

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
def gridsearchwithprecandrecall(model, X_train, y_train, X_test, y_test, tuned_parameters):
    # Set the parameters by cross-validation
    tuned_parameters = [tuned_parameters]
    scores = ['precision', 'recall']

    for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        print()

        clf = GridSearchCV(model, tuned_parameters, cv=2,
                           scoring='%s_macro' % score)
        clf.fit(X_train, y_train)

        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)
        print()
        print("Grid scores on development set:")
        print()
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))
        print()

        print("Detailed classification report:")
        print()
        print("The model is trained on the full development set.")
        print("The scores are computed on the full evaluation set.")
        print()
        y_true, y_predsearch = y_test, clf.predict(X_test)
        print(classification_report(y_true, y_predsearch))
        print()
        return clf

    
def err(clf_stump, clf_err, X_test, y_test, params={}):
    if params != {}:
        clf_err.set_params(**params)
    clf_stump_err = 1.0 - clf_stump.score(X_test, y_test)
    clf_err = 1.0 - clf_err.score(X_test, y_test)
    print('stump error : {0:.2f}; model error : {1:.2f}'.format(clf_stump_err, clf_err))
    return clf_stump_err, clf_err



#IMPORTANT... https://stackoverflow.com/questions/33110973/pass-a-dict-to-scikit-learn-estimator
#clf_dtm_r2.set_params(**clf.best_params_)
#clf_dt_r2 = clf_dtm_r2.fit(X_primercounts2, train2.category)

from sklearn.metrics import zero_one_loss
from sklearn.ensemble import AdaBoostClassifier
def ADA(model, stump_err, mod_err, X_train, y_train, X_test, y_test, n_estimators = 40, learning_rate = 1.):
    ada_discrete = AdaBoostClassifier(
        base_estimator = model,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        algorithm="SAMME")
    ada_discrete.fit(X_train, y_train)

    fig = plt.figure()
    ax = fig.add_subplot(111)

    ax.plot([1, n_estimators], [stump_err] * 2, 'k-',
            label='Decision Stump Error')
    ax.plot([1, n_estimators], [mod_err] * 2, 'k--',
            label='Decision Tree Error')

    ada_discrete_err = numpy.zeros((n_estimators,))
    for i, y_pred in enumerate(ada_discrete.staged_predict(X_test)):
        ada_discrete_err[i] = zero_one_loss(y_pred, y_test)

    ada_discrete_err_train = numpy.zeros((n_estimators,))
    for i, y_pred in enumerate(ada_discrete.staged_predict(X_train)):
        ada_discrete_err_train[i] = zero_one_loss(y_pred, y_train)

    ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err,
            label='Discrete AdaBoost Test Error',
            color='red')
    ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err_train,
            label='Discrete AdaBoost Train Error',
            color='blue')

    ax.set_ylim((0.0, 1.0))
    ax.set_xlabel('n_estimators')
    ax.set_ylabel('error rate')

    leg = ax.legend(loc='upper right', fancybox=True)
    leg.get_frame().set_alpha(0.7)

    plt.show()
    return ada_discrete


## DATA SPLIT

In [159]:
train3=pd_annotated[['platform', 'alltext', 'jscheck', 'docscheck', 'blogscheck', 'commcheck', 'category']].sample(frac=0.75)
test3=pd_annotated[['platform', 'alltext','jscheck', 'docscheck', 'blogscheck', 'commcheck', 'category']].drop(train3.index)

In [160]:
train3.head(10)

,platform,alltext,jscheck,docscheck,blogscheck,commcheck,category
567,docs.webplatform.org,welcome summary community aims docs source d...,0,1,0,0,DOCS
771,www.facebook.com,je een en om end video te die activeer josep...,0,0,0,0,COMMUITY
403,dl.dropboxusercontent.com,error timer link u dropbox file q8zgw lkj2de...,0,0,0,0,NOCLASS
16,openweathermap.org,wind charts world faq urrent img help images...,0,0,0,0,PACKAGE
151,blog.engineyard.com,engine advantages blog service vpc yard anno...,0,0,1,0,NEWS
61,api.jqueryui.com,interested check new library widgets full ef...,0,0,0,0,PACKAGE
74,jsonplaceholder.typicode.com,posts fake rest api server online testing po...,0,0,0,0,PACKAGE
590,port-80-jico6278fu.treehouse-app.com,error encountered preview active quote works...,0,0,0,0,NOCLASS
280,j4n.co,end jan front system user ui interface blog ...,0,0,0,0,NEWS
177,imperavi.com,charts minimalistic simple developers profes...,0,0,0,0,NEWS


In [161]:
test3.head(10)

,platform,alltext,jscheck,docscheck,blogscheck,commcheck,category
1,jscroll.com,plugin loaded github new com iscroll cubiq e...,0,0,0,0,PACKAGE
5,v4-alpha.getbootstrap.com),,0,0,0,0,PACKAGE
7,crossorigin.me,crossorigin org wind github master calculato...,0,0,0,0,PACKAGE
10,flipclockjs.com,conditionals solutions stash enough many dom...,1,0,0,0,PACKAGE
12,anyorigin.com,jsonp access go policy exterminator watch do...,0,0,0,0,PACKAGE
22,www.clementinejs.com,point simplicity starting fcc full developer...,1,0,0,0,PACKAGE
30,jquery.com,library minified also less gzipped jquery gu...,0,0,0,0,PACKAGE
33,maxcdn.bootstrapcdn.com,font theme awesome alpha min cssy js bootstr...,0,0,0,0,PACKAGE
45,angularjs.org,testability pure templates superheroic depen...,1,0,0,0,PACKAGE
46,www.angular-meteor.com,completely synchronization mongodb apollo au...,0,0,0,0,PACKAGE


## DATA MODELLING 3

In [162]:
X_train, countmod = vectorizer(train3.alltext)
X_trainh = hstacker(X_train, train3[['jscheck', 'docscheck', 'blogscheck', 'commcheck']])
y_train = train3.category
X_test = countmod.transform(test3.alltext)
X_testh = hstacker(X_test, test3[['jscheck', 'docscheck', 'blogscheck', 'commcheck']])
y_test = test3.category

## DECISION TREE WITH ADABOOST, CLASSIFICATION 3

### References:
* http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
* http://scikit-learn.org/stable/modules/tree.html
* https://en.wikipedia.org/wiki/AdaBoost
* http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
* http://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_hastie_10_2.html#sphx-glr-auto-examples-ensemble-plot-adaboost-hastie-10-2-py
* https://stackoverflow.com/questions/32210569/using-gridsearchcv-with-adaboost-and-decisiontreeclassifier
* http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html
* http://scikit-learn.org/stable/tutorial/statistical_inference/model_selection.html

**Also**:
* https://stackoverflow.com/questions/33110973/pass-a-dict-to-scikit-learn-estimator
* http://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/

In [163]:
slump_params = {'max_depth':1, 'min_samples_leaf':1, 'random_state':0}
mod_params = {'min_samples_leaf':2, 'random_state':0}

clf_dt_slump, dt_fit_slump = modelling(DecisionTreeClassifier, X_train, y_train, slump_params)
clf_dt_mod, dt_fit = DT(DecisionTreeClassifier, X_train, y_train, mod_params)

clf_dt_slumph, dt_fit_slumph = DT(DecisionTreeClassifier, X_trainh, y_train, slump_params)
clf_dt_modh, dt_fith = DT(DecisionTreeClassifier, X_trainh, y_train, mod_params)

### Parameter Searching for the Decision Tree

In [164]:
tuned_parameters = {'max_depth': numpy.arange(5,25,2)}

In [165]:
gs_clf_dt_mod = gridsearchwithprecandrecall(clf_dt_mod, X_train, y_train, X_test, y_test, tuned_parameters)

# Tuning hyper-parameters for precision



/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec/anaconda3/lib/python3.5/sit

Best parameters set found on development set:

{'max_depth': 19}

Grid scores on development set:

0.164 (+/-0.030) for {'max_depth': 5}
0.168 (+/-0.002) for {'max_depth': 7}
0.204 (+/-0.077) for {'max_depth': 9}
0.201 (+/-0.079) for {'max_depth': 11}
0.223 (+/-0.081) for {'max_depth': 13}
0.226 (+/-0.084) for {'max_depth': 15}
0.225 (+/-0.084) for {'max_depth': 17}
0.295 (+/-0.069) for {'max_depth': 19}
0.245 (+/-0.032) for {'max_depth': 21}
0.249 (+/-0.014) for {'max_depth': 23}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

             precision    recall  f1-score   support

   BUSINESS       0.20      0.17      0.18         6
   COMMUITY       0.00      0.00      0.00         2
       DOCS       0.29      0.40      0.33         5
  ECOMMERCE       0.00      0.00      0.00         2
       NEWS       0.70      0.65      0.67        85
    NOCLASS       0.11      0.09      0.10        11
     

/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [166]:
gs_clf_dt_modh = gridsearchwithprecandrecall(clf_dt_modh, X_trainh, y_train, X_testh, y_test, tuned_parameters)

/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Tuning hyper-parameters for precision



/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec/anaconda3/lib/python3.5/sit

Best parameters set found on development set:

{'max_depth': 17}

Grid scores on development set:

0.235 (+/-0.032) for {'max_depth': 5}
0.253 (+/-0.010) for {'max_depth': 7}
0.305 (+/-0.107) for {'max_depth': 9}
0.284 (+/-0.034) for {'max_depth': 11}
0.313 (+/-0.033) for {'max_depth': 13}
0.296 (+/-0.068) for {'max_depth': 15}
0.321 (+/-0.027) for {'max_depth': 17}
0.250 (+/-0.108) for {'max_depth': 19}
0.262 (+/-0.108) for {'max_depth': 21}
0.266 (+/-0.140) for {'max_depth': 23}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

             precision    recall  f1-score   support

   BUSINESS       0.20      0.17      0.18         6
   COMMUITY       0.00      0.00      0.00         2
       DOCS       0.00      0.00      0.00         5
  ECOMMERCE       0.00      0.00      0.00         2
       NEWS       0.70      0.65      0.67        85
    NOCLASS       0.00      0.00      0.00        11
     

/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### Error Analysis

In [167]:
###a different parameter to be adjusted? use gs_clf!!!


In [1]:
clf_dt_stump_err, clf_dt_mod_err = err(clf_dt_slump, clf_dt_mod, X_test, y_test, gs_clf_dt_mod.best_params_)
clf_dt_stump_errh, clf_dt_mod_errh = err(clf_dt_slumph, clf_dt_modh, X_testh, y_test, gs_clf_dt_modh.best_params_)

NameError: name 'err' is not defined

### Ada Boost

In [ ]:
clf_adadt_mod = ADA(clf_dt_mod, clf_dt_stump_err, clf_dt_mod_err, X_train, y_train, X_test, y_test, n_estimators = 40, learning_rate = .5)
clf_adadt_modh = ADA(clf_dt_modh, clf_dt_stump_errh, clf_dt_mod_errh, X_trainh, y_train, X_testh, y_test, n_estimators = 40, learning_rate = .5)

In [ ]:
STOP HERE!

## RANDOM FOREST WITH ADABOOST, CLASSIFICATION 3

### References:
* http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
* http://scikit-learn.org/stable/modules/tree.html
* https://en.wikipedia.org/wiki/AdaBoost
* http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
* http://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_hastie_10_2.html#sphx-glr-auto-examples-ensemble-plot-adaboost-hastie-10-2-py
* https://stackoverflow.com/questions/32210569/using-gridsearchcv-with-adaboost-and-decisiontreeclassifier
* http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html
* http://scikit-learn.org/stable/tutorial/statistical_inference/model_selection.html

**Also**:
* https://stackoverflow.com/questions/33110973/pass-a-dict-to-scikit-learn-estimator
* http://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/

In [ ]:
slump_params = {'max_depth':1, 'min_samples_leaf':1, 'random_state':0}
mod_params = {'min_samples_leaf':2, 'random_state':0}


clf_rf_slump, rf_fit_slump = modelling(RandomForestClassifier, X_train, y_train, slump_params)
clf_rf_mod, rf_fit = DT(RandomForestClassifier, X_train, y_train, mod_params)

clf_rf_slumph, rf_fit_slumph = DT(RandomForestClassifier, X_trainh, y_train, slump_params)
clf_rf_modh, rf_fith = DT(RandomForestClassifier, X_trainh, y_train, mod_params)

### Parameter Searching for the Decision Tree

In [ ]:
tuned_parameters = {'max_depth': numpy.arange(5,25,2), 'n_estimators': numpy.arrange(5,30,5)}

In [ ]:
gs_clf_rf_mod = gridsearchwithprecandrecall(clf_rf_mod, X_train, y_train, X_test, y_test, tuned_parameters)

In [ ]:
gs_clf_rf_modh = gridsearchwithprecandrecall(clf_rf_modh, X_trainh, y_train, X_testh, y_test, tuned_parameters)

### Error Analysis

In [ ]:
###a different parameter to be adjusted? use gs_clf!!!


In [ ]:
clf_rf_stump_err, clf_rf_mod_err = err(clf_rf_slump, clf_rf_mod, X_test, y_test, gs_clf_rf_mod.best_params_)
clf_rf_stump_errh, clf_rf_mod_errh = err(clf_rf_slumph, clf_rf_modh, X_testh, y_test, gs_clf_rf_modh.best_params_)

### Ada Boost

In [ ]:
clf_adarf_mod = ADA(clf_rf_mod, clf_rf_stump_err, clf_rf_mod_err, X_train, y_train, X_test, y_test, n_estimators = 40, learning_rate = .5)
clf_adarf_modh = ADA(clf_rf_modh, clf_rf_stump_errh, clf_rf_mod_errh, X_trainh, y_train, X_testh, y_test, n_estimators = 40, learning_rate = .5)

In [ ]:
STOP HERE!

In [ ]:
STOP HERE!

In [ ]:
STOP HERE!

## DATA MODELLING - VECTOR MODEL 3

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect_r3 = CountVectorizer(ngram_range=(1,2))
X_primercounts3 = count_vect_r3.fit_transform(train3.alltext)
X_primercounts3.shape

In [ ]:
type(X_primercounts3)

In [ ]:
from scipy.sparse import csr_matrix, hstack
import copy
X_primercounts3h = hstack((copy.copy(X_primercounts3), train3[['jscheck', 'docscheck', 'blogscheck', 'commcheck']]))


In [ ]:
X_primercounts3.shape

In [ ]:
X_primercounts3h.shape

## DECISION TREE WITH ADABOOST, CLASSIFICATION 3

### References:
* http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
* http://scikit-learn.org/stable/modules/tree.html
* https://en.wikipedia.org/wiki/AdaBoost
* http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
* http://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_hastie_10_2.html#sphx-glr-auto-examples-ensemble-plot-adaboost-hastie-10-2-py
* https://stackoverflow.com/questions/32210569/using-gridsearchcv-with-adaboost-and-decisiontreeclassifier
* http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html
* http://scikit-learn.org/stable/tutorial/statistical_inference/model_selection.html

**Also**:
* https://stackoverflow.com/questions/33110973/pass-a-dict-to-scikit-learn-estimator
* http://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/

### Models

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import zero_one_loss
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
clf_dtm_r3_stump = DecisionTreeClassifier(max_depth=1, min_samples_leaf=1, random_state=0)
clf_dtm_r3 = DecisionTreeClassifier(min_samples_leaf=2, random_state=0)

clf_dt_r3_stump = clf_dtm_r3_stump.fit(X_primercounts3, train3.category)
clf_dt_r3 = clf_dtm_r3.fit(X_primercounts3, train3.category)


clf_dtm_r3_stumph = DecisionTreeClassifier(max_depth=1, min_samples_leaf=1, random_state=0)
clf_dtm_r3h = DecisionTreeClassifier(min_samples_leaf=2, random_state=0)

clf_dt_r3_stumph = clf_dtm_r3_stumph.fit(X_primercounts3h, train3.category)
clf_dt_r3h = clf_dtm_r3h.fit(X_primercounts3h, train3.category)

### Exploration of the Classification

In [ ]:
X_test3counts_stump = count_vect_r3.transform(test3.alltext)
predicted_dt_r3_stump = clf_dt_r3_stump.predict(X_test3counts_stump)

from scipy.sparse import csr_matrix, hstack
X_test3counts_stumph = hstack((X_test3counts_stump, test3[['jscheck', 'docscheck', 'blogscheck', 'commcheck']]))
predicted_dt_r3_stumph = clf_dt_r3_stumph.predict(X_test3counts_stumph)

In [ ]:
#for platform, category in zip(test.platform, predicted_dt_r1_stump):
#    print('%r => %s' % (platform, category))

In [ ]:
X_test3counts = count_vect_r3.transform(test3.alltext)
predicted_dt_r3 = clf_dt_r3.predict(X_test3counts)

from scipy.sparse import csr_matrix, hstack
X_test3countsh = hstack((X_test3counts, test3[['jscheck', 'docscheck', 'blogscheck', 'commcheck']]))
predicted_dt_r3h = clf_dt_r3h.predict(X_test3countsh)

In [ ]:
for plat, category in zip(zip(test3.platform,test3.category), predicted_dt_r3):
    print('%r => %s => %s' % (plat[0], plat[1], category))

### Parameter Searching for the Decision Tree

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the parameters by cross-validation
tuned_parameters = [{'max_depth': numpy.arange(5,25,2)}]
scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(clf_dtm_r3, tuned_parameters, cv=2,
                       scoring='%s_macro' % score)
    clf.fit(X_primercounts3, train3.category)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_predsearch = test3.category, clf.predict(X_test3counts)
    print(classification_report(y_true, y_predsearch))
    print()


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the parameters by cross-validation
tuned_parameters = [{'max_depth': numpy.arange(5,25,2)}]
scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(clf_dtm_r3h, tuned_parameters, cv=2,
                       scoring='%s_macro' % score)
    clf.fit(X_primercounts3h, train3.category)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_predsearch = test3.category, clf.predict(X_test3countsh)
    print(classification_report(y_true, y_predsearch))
    print()

In [ ]:
#X_test3counts_stump = count_vect_r3.transform(test3.alltext)

#from scipy.sparse import csr_matrix, hstack
#X_test3counts_stump = hstack((X_test3counts_stump, test3[['jscheck', 'docscheck', 'blogscheck', 'commcheck']]))

clf_dt_r3_stump_err = 1.0 - clf_dt_r3_stump.score(X_test3counts_stump, test3.category)

#X_test3count = count_vect_r3.transform(test3.alltext)

#from scipy.sparse import csr_matrix, hstack
#X_test3counts = hstack((X_test3counts, test3[['jscheck', 'docscheck', 'blogscheck', 'commcheck']]))

clf_dt_r3_err = 1.0 - clf.score(X_test3counts, test3.category)

In [ ]:
print(clf_dt_r3_stump_err,clf_dt_r3_err)

In [ ]:
print(clf.best_params_)

In [ ]:
#clf_dtm_r3 = DecisionTreeClassifier(min_samples_leaf=2, random_state=0)
clf_dtm_r3.set_params(**{'max_depth': 13})
clf_dt_r3 = clf_dtm_r3.fit(X_primercounts3, train3.category)

In [ ]:
clf_dt_r3_err = 1.0 - clf_dt_r3.score(X_test3counts, test3.category)

In [ ]:
print(clf_dt_r3_stump_err,clf_dt_r3_err)

### AdaBoost

In [ ]:
n_estimators = 40
# A learning rate of 1. may not be optimal for both SAMME and SAMME.R
learning_rate = .2


#IMPORTANT... https://stackoverflow.com/questions/33110973/pass-a-dict-to-scikit-learn-estimator
#clf_dtm_r2.set_params(**clf.best_params_)
#clf_dt_r2 = clf_dtm_r2.fit(X_primercounts2, train2.category)

ada_discrete = AdaBoostClassifier(
    base_estimator = clf_dt_r3,
    learning_rate=learning_rate,
    n_estimators=n_estimators,
    algorithm="SAMME")
ada_discrete.fit(X_primercounts3, train3.category)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot([1, n_estimators], [clf_rf_r3_stump_err] * 2, 'k-',
        label='Decision Stump Error')
ax.plot([1, n_estimators], [clf_rf_r3_err] * 2, 'k--',
        label='Decision Tree Error')

ada_discrete_err = numpy.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_test3counts)):
    ada_discrete_err[i] = zero_one_loss(y_pred, test3.category)

ada_discrete_err_train = numpy.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_primercounts3)):
    ada_discrete_err_train[i] = zero_one_loss(y_pred, train3.category)

ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err,
        label='Discrete AdaBoost Test Error',
        color='red')
ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err_train,
        label='Discrete AdaBoost Train Error',
        color='blue')

ax.set_ylim((0.0, 1.0))
ax.set_xlabel('n_estimators')
ax.set_ylabel('error rate')

leg = ax.legend(loc='upper right', fancybox=True)
leg.get_frame().set_alpha(0.7)

plt.show()

In [ ]:
predicted_ada_r3 = ada_discrete.predict(X_test3counts)

In [ ]:
for *bcategory, category in zip(zip(zip(test3.platform,test3.category), best_categories), predicted_ada_r3):
    plat = bcategory[0][0]
    bcategory = bcategory[0][1]
    print('%r => %s => %s => %s' % (plat[0], plat[1], bcategory.lower(), category))

In [ ]:
print(classification_report(test3.category, predicted_ada_r3))

In [ ]:
n_estimators = 40
# A learning rate of 1. may not be optimal for both SAMME and SAMME.R
learning_rate = .2


#IMPORTANT... https://stackoverflow.com/questions/33110973/pass-a-dict-to-scikit-learn-estimator
#clf_dtm_r2.set_params(**clf.best_params_)
#clf_dt_r2 = clf_dtm_r2.fit(X_primercounts2, train2.category)

ada_discrete = AdaBoostClassifier(
    base_estimator = clf_dt_r3h,
    learning_rate=learning_rate,
    n_estimators=n_estimators,
    algorithm="SAMME")
ada_discrete.fit(X_primercounts3h, train3.category)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot([1, n_estimators], [clf_dt_r3_stump_err] * 2, 'k-',
        label='Decision Stump Error')
ax.plot([1, n_estimators], [clf_rf_r3_err] * 2, 'k--',
        label='Decision Tree Error')

ada_discrete_err = numpy.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_test3counts)):
    ada_discrete_err[i] = zero_one_loss(y_pred, test3.category)

ada_discrete_err_train = numpy.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_primercounts3)):
    ada_discrete_err_train[i] = zero_one_loss(y_pred, train3.category)

ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err,
        label='Discrete AdaBoost Test Error',
        color='red')
ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err_train,
        label='Discrete AdaBoost Train Error',
        color='blue')

ax.set_ylim((0.0, 1.0))
ax.set_xlabel('n_estimators')
ax.set_ylabel('error rate')

leg = ax.legend(loc='upper right', fancybox=True)
leg.get_frame().set_alpha(0.7)

plt.show()

In [ ]:
predicted_ada_r3 = ada_discrete.predict(X_test3counts)

In [ ]:
for *bcategory, category in zip(zip(zip(test3.platform,test3.category), best_categories), predicted_ada_r3):
    plat = bcategory[0][0]
    bcategory = bcategory[0][1]
    print('%r => %s => %s => %s' % (plat[0], plat[1], bcategory.lower(), category))

In [ ]:
print(classification_report(test3.category, predicted_ada_r3))

In [ ]:
here stop

## RANDOM FOREST, CLASSIFICATION 2

### References:
* http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
* http://scikit-learn.org/stable/modules/ensemble.html#forest

### Models

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import zero_one_loss

In [ ]:
n_estimators = 25
# A learning rate of 1. may not be optimal for both SAMME and SAMME.R
#learning_rate = 1.

clf_rfm_r3_stump = RandomForestClassifier(max_depth=1, min_samples_leaf=1, n_estimators=n_estimators, random_state=0)
clf_rf_r3_stump = clf_rfm_r3_stump.fit(X_primercounts3, train3.category)

clf_rfm_r3 = RandomForestClassifier(max_depth=20, min_samples_leaf=2, n_estimators=n_estimators, random_state=0)
clf_rf_r3 = clf_rfm_r3.fit(X_primercounts3, train3.category)

### Classification

In [ ]:
#X_test3counts_stump = count_vect_r3.transform(test3.alltext)
predicted_rf_r3_stump = clf_rf_r3_stump.predict(X_test3counts_stump)

In [ ]:
for platform, category in zip(test2.platform, predicted_rf_r3_stump):
    print('%r => %s' % (platform, category))

In [ ]:
#X_test3counts = count_vect_r3.transform(test3.alltext)
predicted_rf_r3 = clf_rf_r3.predict(X_test3counts)

In [ ]:
for plat, category1, category2 in zip(zip(test3.platform,test3.category), predicted_rf_r3, predicted_dt_r3):
    best_category = ''
    print('%r => %s => %s => %s => %s' % (plat[0], plat[1].lower(), category1, category2, best_category.lower()))

In [ ]:
#X_test2counts_stump = count_vect_r2.transform(test2.alltext)
clf_rf_r3_stump_err = 1.0 - clf_rf_r3_stump.score(X_test3counts_stump, test3.category)
#X_test2count = count_vect_r2.transform(test2.alltext)
clf_rf_r3_err = 1.0 - clf_rf_r3.score(X_test3counts, test3.category)

clf_rf_r3_bc_err = 1.0 - clf_rf_r3.score(X_test3counts, best_categories)

In [ ]:
print(clf_rf_r3_stump_err,clf_rf_r3_err, clf_rf_r3_bc_err)

In [ ]:
print(classification_report(test3.category, best_categories))

In [ ]:
print(classification_report(test3.category, predicted_rf_r3))

### AdaBoost

In [ ]:
n_estimators = 40
# A learning rate of 1. may not be optimal for both SAMME and SAMME.R
learning_rate = .2


#IMPORTANT... https://stackoverflow.com/questions/33110973/pass-a-dict-to-scikit-learn-estimator
#clf_dtm_r2.set_params(**clf.best_params_)
#clf_dt_r2 = clf_dtm_r2.fit(X_primercounts2, train2.category)

ada_discrete = AdaBoostClassifier(
    base_estimator = clf_rf_r3,
    learning_rate=learning_rate,
    n_estimators=n_estimators,
    algorithm="SAMME")
ada_discrete.fit(X_primercounts3, train3.category)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot([1, n_estimators], [clf_rf_r3_stump_err] * 2, 'k-',
        label='Decision Stump Error')
ax.plot([1, n_estimators], [clf_rf_r3_err] * 2, 'k--',
        label='Decision Tree Error')

ada_discrete_err = numpy.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_test3counts)):
    ada_discrete_err[i] = zero_one_loss(y_pred, test3.category)

ada_discrete_err_train = numpy.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_primercounts3)):
    ada_discrete_err_train[i] = zero_one_loss(y_pred, train3.category)

ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err,
        label='Discrete AdaBoost Test Error',
        color='red')
ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err_train,
        label='Discrete AdaBoost Train Error',
        color='blue')

ax.set_ylim((0.0, 1.0))
ax.set_xlabel('n_estimators')
ax.set_ylabel('error rate')

leg = ax.legend(loc='upper right', fancybox=True)
leg.get_frame().set_alpha(0.7)

plt.show()

In [ ]:
predicted_ada_r3 = ada_discrete.predict(X_test3counts)

In [ ]:
for *bcategory, category in zip(zip(zip(test3.platform,test3.category), best_categories), predicted_ada_r3):
    plat = bcategory[0][0]
    bcategory = bcategory[0][1]
    print('%r => %s => %s => %s' % (plat[0], plat[1], bcategory.lower(), category))

In [ ]:
newbest_categories = []
for *bcategory, category in zip(zip(zip(test3.platform,test3.category), best_categories), predicted_ada_r3):
    plat = bcategory[0][0]
    bcategory = bcategory[0][1]
    if bcategory == 'TRAINING' and category != 'NOCLASS':
        newbest_categories.append('TRAINING')
    else:
        newbest_categories.append(category)
    
    if category == 'REPL':
        print('%r => %s => %s => %s' % (plat[0], plat[1], bcategory.lower(), category))

In [ ]:
print(classification_report(test3.category, predicted_ada_r3))

In [ ]:
print(classification_report(test3.category, newbest_categories))

## Data Saving

# ANALYSIS 3

## DATA SPLIT

In [ ]:
train3=pd_annotated2[['platform', 'alltext','category']].sample(frac=0.75,random_state=0)
test3=pd_annotated2[['platform', 'alltext','category']].drop(train3.index)

In [ ]:
train3.head(10)

## DATA MODELLING - VECTOR MODEL 2

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect_r2 = CountVectorizer(ngram_range=(1,2))
X_primercounts2 = count_vect_r2.fit_transform(train2.alltext)
X_primercounts2.shape

## DECISION TREE WITH ADABOOST, CLASSIFICATION 2

### References:
* http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
* http://scikit-learn.org/stable/modules/tree.html
* https://en.wikipedia.org/wiki/AdaBoost
* http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
* http://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_hastie_10_2.html#sphx-glr-auto-examples-ensemble-plot-adaboost-hastie-10-2-py
* https://stackoverflow.com/questions/32210569/using-gridsearchcv-with-adaboost-and-decisiontreeclassifier
* http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html
* http://scikit-learn.org/stable/tutorial/statistical_inference/model_selection.html

**Also**:
* https://stackoverflow.com/questions/33110973/pass-a-dict-to-scikit-learn-estimator
* http://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/

### Models

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import zero_one_loss
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
clf_dtm_r2_stump = DecisionTreeClassifier(max_depth=1, min_samples_leaf=1, random_state=0)
clf_dt_r2_stump = clf_dtm_r2_stump.fit(X_primercounts2, train2.category)

clf_dtm_r2 = DecisionTreeClassifier(min_samples_leaf=2, random_state=0)
clf_dt_r2 = clf_dtm_r2.fit(X_primercounts2, train2.category)

### Exploration of the Classification

In [ ]:
X_test2counts_stump = count_vect_r2.transform(test2.alltext)
predicted_dt_r2_stump = clf_dt_r2_stump.predict(X_test2counts_stump)

In [ ]:
#for platform, category in zip(test.platform, predicted_dt_r1_stump):
#    print('%r => %s' % (platform, category))

In [ ]:
X_test2counts = count_vect_r2.transform(test2.alltext)
predicted_dt_r2 = clf_dt_r2.predict(X_test2counts)

In [ ]:
for plat, category in zip(zip(test2.platform,test2.category), predicted_dt_r2):
    print('%r => %s => %s' % (plat[0], plat[1], category))

### Parameter Searching for the Decision Tree

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the parameters by cross-validation
tuned_parameters = [{'max_depth': numpy.arange(5,25,2)}]
scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(clf_dtm_r2, tuned_parameters, cv=2,
                       scoring='%s_macro' % score)
    clf.fit(X_primercounts2, train2.category)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_predsearch = test2.category, clf.predict(X_test2counts)
    print(classification_report(y_true, y_predsearch))
    print()

In [ ]:
X_test2counts_stump = count_vect_r2.transform(test2.alltext)
clf_dt_r2_stump_err = 1.0 - clf_dt_r2_stump.score(X_test2counts_stump, test2.category)
X_test2count = count_vect_r2.transform(test2.alltext)
clf_dt_r2_err = 1.0 - clf.score(X_test2counts, test2.category)

In [ ]:
print(clf_dt_r2_stump_err,clf_dt_r2_err)

In [ ]:
print(clf.best_params_)

### AdaBoost

In [ ]:
n_estimators = 40
# A learning rate of 1. may not be optimal for both SAMME and SAMME.R
learning_rate = .5


#IMPORTANT... https://stackoverflow.com/questions/33110973/pass-a-dict-to-scikit-learn-estimator
clf_dtm_r2.set_params(**clf.best_params_)
clf_dt_r2 = clf_dtm_r2.fit(X_primercounts2, train2.category)

ada_discrete = AdaBoostClassifier(
    base_estimator = clf_dtm_r2,
    learning_rate=learning_rate,
    n_estimators=n_estimators,
    algorithm="SAMME")
ada_discrete.fit(X_primercounts2, train2.category)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot([1, n_estimators], [clf_dt_r2_stump_err] * 2, 'k-',
        label='Decision Stump Error')
ax.plot([1, n_estimators], [clf_dt_r2_err] * 2, 'k--',
        label='Decision Tree Error')

ada_discrete_err = numpy.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_test2counts)):
    ada_discrete_err[i] = zero_one_loss(y_pred, test2.category)

ada_discrete_err_train = numpy.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_primercounts2)):
    ada_discrete_err_train[i] = zero_one_loss(y_pred, train2.category)

ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err,
        label='Discrete AdaBoost Test Error',
        color='red')
ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err_train,
        label='Discrete AdaBoost Train Error',
        color='blue')

ax.set_ylim((0.0, 1.0))
ax.set_xlabel('n_estimators')
ax.set_ylabel('error rate')

leg = ax.legend(loc='upper right', fancybox=True)
leg.get_frame().set_alpha(0.7)

plt.show()

In [ ]:
Stop HERE!

In [ ]:
with open(os.getcwd()+'/data/adaboostmodel1.pkl', 'bw') as mod:
    pickle.dump([train, test, train.alltext, train.category, count_vect_r1, X_primercounts, clf_dtm_r1, clf_dt_r1, ada_discrete, n_estimators, learning_rate], mod)

## Classifying unlabelled records

In [ ]:
X_unclass1counts = count_vect_r1.transform(pd_notated.alltext)
#normalized_X_testround1_counts = sklearn.feature_extraction.text.TfidfTransformer(norm='l2').fit_transform(X_testround1_counts)
predicted_dtada_r1 = ada_discrete.predict(X_unclass1counts)

In [ ]:
for platform, category in zip(pd_notated.platform, predicted_dtada_r1):
    print('%r => %s' % (platform, category))

In [ ]:
with open(os.getcwd()+'/data/abadt_1.csv', 'w') as outfile:
    writer = csv.writer(outfile, delimiter=';', quotechar="'")
    writer.writerow(['platform','assignedcategory','checked','category'])
    for platform, category in zip(pd_notated.platform, predicted_dtada_r1):
        #print('%r => %s' % (platform, category))
        writer.writerow([platform,category,,])

In [ ]:
here stop

# DUMPED!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_predict
from sklearn import metrics

## RANDOM FOREST, CLASSIFICATION 1

### References:
* http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
* http://scikit-learn.org/stable/modules/ensemble.html#forest

### Models

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import zero_one_loss

In [ ]:
#n_estimators = 25
# A learning rate of 1. may not be optimal for both SAMME and SAMME.R
#learning_rate = 1.

clf_rfm_r2_stump = RandomForestClassifier(max_depth=1, min_samples_leaf=1, n_estimators=n_estimators, random_state=0)
clf_rf_r2_stump = clf_rfm_r2_stump.fit(X_primercounts2, train2.category)

clf_rfm_r2 = RandomForestClassifier(max_depth=20, min_samples_leaf=2, n_estimators=n_estimators, random_state=0)
clf_rf_r2 = clf_rfm_r2.fit(X_primercounts2, train2.category)

### Classification

In [ ]:
#X_test1counts_stump = count_vect_r2.transform(test.alltext)
predicted_rf_r2_stump = clf_rf_r2_stump.predict(X_test2counts_stump)

In [ ]:
for platform, category in zip(test2.platform, predicted_rf_r2_stump):
    print('%r => %s' % (platform, category))

In [ ]:
#X_test2counts = count_vect_r2.transform(test2.alltext)
predicted_rf_r2 = clf_rf_r2.predict(X_test2counts)

In [ ]:
best_categories = []
for plat, category1, category2 in zip(zip(test2.platform,test2.category), predicted_rf_r2, predicted_dt_r2):
    best_category = ''
    if category1 == category2:
        best_category = category1
    else:
        if category1 == 'NEWS' and category2 == 'TRAINING':
            best_category = category1

        else:
            best_category = category2
    
    if category1 != category2 and len(plat[0].split('.')) > 1:
        if category2 in ['NOCLASS', 'BUSINESS', 'DOCS', 'PAAS', 'NEWS']:
            #print(plat[0])
            if len(plat[0].split('.')[-2]) > 2:
                #print(plat[0].split('.')[-2])
                if plat[0].split('.')[-2][-2:] == 'js':
                    #print(len(plat[0].split('.')))
                    #print(plat[0].split('.')[-2])
                    best_category = 'PACKAGE'
    
    best_categories.append(best_category)
    print('%r => %s => %s => %s => %s' % (plat[0], plat[1].lower(), category1, category2, best_category.lower()))

In [ ]:
#X_test2counts_stump = count_vect_r2.transform(test2.alltext)
clf_rf_r2_stump_err = 1.0 - clf_rf_r2_stump.score(X_test2counts_stump, test2.category)
#X_test2count = count_vect_r2.transform(test2.alltext)
clf_rf_r2_err = 1.0 - clf_rf_r2.score(X_test2counts, test2.category)

clf_rf_r2_bc_err = 1.0 - clf_rf_r2.score(X_test2counts, best_categories)

In [ ]:
print(clf_rf_r2_stump_err,clf_rf_r2_err, clf_rf_r2_bc_err)

In [ ]:
print(classification_report(test2.category, best_categories))

### AdaBoost

In [ ]:
n_estimators = 40
# A learning rate of 1. may not be optimal for both SAMME and SAMME.R
learning_rate = .2


#IMPORTANT... https://stackoverflow.com/questions/33110973/pass-a-dict-to-scikit-learn-estimator
#clf_dtm_r2.set_params(**clf.best_params_)
#clf_dt_r2 = clf_dtm_r2.fit(X_primercounts2, train2.category)

ada_discrete = AdaBoostClassifier(
    base_estimator = clf_rf_r2,
    learning_rate=learning_rate,
    n_estimators=n_estimators,
    algorithm="SAMME")
ada_discrete.fit(X_primercounts2, train2.category)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot([1, n_estimators], [clf_rf_r2_stump_err] * 2, 'k-',
        label='Decision Stump Error')
ax.plot([1, n_estimators], [clf_rf_r2_err] * 2, 'k--',
        label='Decision Tree Error')

ada_discrete_err = numpy.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_test2counts)):
    ada_discrete_err[i] = zero_one_loss(y_pred, test2.category)

ada_discrete_err_train = numpy.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_primercounts2)):
    ada_discrete_err_train[i] = zero_one_loss(y_pred, train2.category)

ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err,
        label='Discrete AdaBoost Test Error',
        color='red')
ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err_train,
        label='Discrete AdaBoost Train Error',
        color='blue')

ax.set_ylim((0.0, 1.0))
ax.set_xlabel('n_estimators')
ax.set_ylabel('error rate')

leg = ax.legend(loc='upper right', fancybox=True)
leg.get_frame().set_alpha(0.7)

plt.show()

In [ ]:
predicted_ada_r2 = ada_discrete.predict(X_test2counts)

In [ ]:
newbest_categories = []
for *bcategory, category in zip(zip(zip(test2.platform,test2.category), best_categories), predicted_ada_r2):
    plat = bcategory[0][0]
    bcategory = bcategory[0][1]
    if bcategory == 'TRAINING' and category != 'NOCLASS':
        newbest_categories.append('TRAINING')
    else:
        newbest_categories.append(category)
    
    if category == 'REPL':
        print('%r => %s => %s => %s' % (plat[0], plat[1], bcategory.lower(), category))

In [ ]:
print(classification_report(test2.category, predicted_ada_r2))

In [ ]:
print(classification_report(test2.category, newbest_categories))

# ANALYSIS ROUND 1

## References:
* http://scikit-learn.org/stable/modules/cross_validation.html
* https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas

## DATA SPLIT

In [ ]:
#from sklearn.model_selection import train_test_split
#xtrain, xtest, ytrain, ytest = train_test_split(pd_annotated.alltext, pd_annotated.category, test_size=0.1, random_state=0)

In [ ]:
train=pd_annotated[['platform', 'alltext','category']].sample(frac=0.75,random_state=0)
test=pd_annotated[['platform', 'alltext','category']].drop(train.index)

In [ ]:
train.head(10)

## DATA MODELLING - VECTOR MODEL 1

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect_r1 = CountVectorizer(ngram_range=(1,2))
X_primercounts = count_vect_r1.fit_transform(train.alltext)
X_primercounts.shape

## DECISION TREE WITH ADABOOST, CLASSIFICATION 1

### References:
* http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
* http://scikit-learn.org/stable/modules/tree.html
* https://en.wikipedia.org/wiki/AdaBoost
* http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
* http://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_hastie_10_2.html#sphx-glr-auto-examples-ensemble-plot-adaboost-hastie-10-2-py
* https://stackoverflow.com/questions/32210569/using-gridsearchcv-with-adaboost-and-decisiontreeclassifier
* http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html
* http://scikit-learn.org/stable/tutorial/statistical_inference/model_selection.html

**Also**:
* https://stackoverflow.com/questions/33110973/pass-a-dict-to-scikit-learn-estimator
* http://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/

### Models

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import zero_one_loss
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
clf_dtm_r1_stump = DecisionTreeClassifier(max_depth=1, min_samples_leaf=1, random_state=0)
clf_dt_r1_stump = clf_dtm_r1_stump.fit(X_primercounts, train.category)

clf_dtm_r1 = DecisionTreeClassifier(min_samples_leaf=2, random_state=0)
clf_dt_r1 = clf_dtm_r1.fit(X_primercounts, train.category)

### Exploration of the Classification

In [ ]:
X_test1counts_stump = count_vect_r1.transform(test.alltext)
predicted_dt_r1_stump = clf_dt_r1_stump.predict(X_test1counts_stump)

In [ ]:
#for platform, category in zip(test.platform, predicted_dt_r1_stump):
#    print('%r => %s' % (platform, category))

In [ ]:
X_test1counts = count_vect_r1.transform(test.alltext)
predicted_dt_r1 = clf_dt_r1.predict(X_test1counts)

In [ ]:
#for platform, category in zip(test.platform, predicted_dt_r1):
#    print('%r => %s' % (platform, category))

### Parameter Searching for the Decision Tree

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the parameters by cross-validation
tuned_parameters = [{'max_depth': numpy.arange(5,25,5)}]
scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(clf_dtm_r1, tuned_parameters, cv=2,
                       scoring='%s_macro' % score)
    clf.fit(X_primercounts, train.category)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_predsearch = test.category, clf.predict(X_test1counts)
    print(classification_report(y_true, y_predsearch))
    print()

In [ ]:
X_test1counts_stump = count_vect_r1.transform(test.alltext)
clf_dt_r1_stump_err = 1.0 - clf_dt_r1_stump.score(X_test1counts_stump, test.category)
X_test1count = count_vect_r1.transform(test.alltext)
clf_err = 1.0 - clf.score(X_test1counts, test.category)

In [ ]:
print(clf_dt_r1_stump_err,clf_dt_r1_err)

In [ ]:
print(clf.best_params_)

### AdaBoost

In [ ]:
n_estimators = 40
# A learning rate of 1. may not be optimal for both SAMME and SAMME.R
learning_rate = 1.


#IMPORTANT... https://stackoverflow.com/questions/33110973/pass-a-dict-to-scikit-learn-estimator
clf_dtm_r1.set_params(**clf.best_params_)
clf_dt_r1 = clf_dtm_r1.fit(X_primercounts, train.category)

ada_discrete = AdaBoostClassifier(
    base_estimator = clf_dtm_r1,
    learning_rate=learning_rate,
    n_estimators=n_estimators,
    algorithm="SAMME")
ada_discrete.fit(X_primercounts, train.category)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot([1, n_estimators], [clf_dt_r1_stump_err] * 2, 'k-',
        label='Decision Stump Error')
ax.plot([1, n_estimators], [clf_dt_r1_err] * 2, 'k--',
        label='Decision Tree Error')

ada_discrete_err = numpy.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_test1counts)):
    ada_discrete_err[i] = zero_one_loss(y_pred, test.category)

ada_discrete_err_train = numpy.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_primercounts)):
    ada_discrete_err_train[i] = zero_one_loss(y_pred, train.category)

ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err,
        label='Discrete AdaBoost Test Error',
        color='red')
ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err_train,
        label='Discrete AdaBoost Train Error',
        color='blue')

ax.set_ylim((0.0, 1.0))
ax.set_xlabel('n_estimators')
ax.set_ylabel('error rate')

leg = ax.legend(loc='upper right', fancybox=True)
leg.get_frame().set_alpha(0.7)

plt.show()

In [ ]:
with open(os.getcwd()+'/data/adaboostmodel1.pkl', 'bw') as mod:
    pickle.dump([train, test, train.alltext, train.category, count_vect_r1, X_primercounts, clf_dtm_r1, clf_dt_r1, ada_discrete, n_estimators, learning_rate], mod)

## Classifying unlabelled records

In [ ]:
X_unclass1counts = count_vect_r1.transform(pd_notated.alltext)
#normalized_X_testround1_counts = sklearn.feature_extraction.text.TfidfTransformer(norm='l2').fit_transform(X_testround1_counts)
predicted_dtada_r1 = ada_discrete.predict(X_unclass1counts)

In [ ]:
for platform, category in zip(pd_notated.platform, predicted_dtada_r1):
    print('%r => %s' % (platform, category))

In [ ]:
with open(os.getcwd()+'/data/abadt_1.csv', 'w') as outfile:
    writer = csv.writer(outfile, delimiter=';', quotechar="'")
    writer.writerow(['platform','assignedcategory','checked','category'])
    for platform, category in zip(pd_notated.platform, predicted_dtada_r1):
        #print('%r => %s' % (platform, category))
        writer.writerow([platform,category,,])

In [ ]:
STOP HERE!!

# POST-ANALYSIS ROUND 1

Data preparation after the manual supervision of the classes

### Python Libraries

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import zero_one_loss
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

## DATA PREPARATION

In [ ]:
with open(os.getcwd()+'/data/adaboostmodel1.pkl', 'br') as mod:
    train, test, train.alltext, train.category, count_vect_r1, X_primercounts, clf_dtm_r1, clf_dt_r1, ada_discrete, n_estimators, learning_rate = pickle.load(mod)

In [ ]:
test.head()

In [ ]:
X_unclass1counts = count_vect_r1.transform(pd_notated.alltext)

In [ ]:
pd_notatedready_r1 = pandas.read_csv(open(os.getcwd()+'/data/abadt_1.csv', 'r'), sep=';', quotechar="'" )

In [ ]:
pd_notatedready_r1.loc[pd_notatedready_r1['checked'] != -1, :].shape

In [ ]:
pd_notatedready_r1.shape

## EVALUATION

In [ ]:
print(classification_report(pd_notatedready_r1.loc[pd_notatedready_r1['checked'] != -1, 'category'], pd_notatedready_r1.loc[pd_notatedready_r1['checked'] != -1, 'assignedcategory']))